## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Neijiang,CN,29.5835,105.0622,73.20,87,100,4.14,overcast clouds,2022-06-13 00:50:06
1,1,Punta Arenas,CL,-53.1500,-70.9167,46.51,81,75,6.91,light rain,2022-06-13 00:48:35
2,2,Kieta,PG,-6.2167,155.6333,85.26,66,18,5.68,few clouds,2022-06-13 00:50:07
3,3,Adrar,MR,20.5022,-10.0711,89.20,26,1,11.90,clear sky,2022-06-13 00:50:07
4,4,Barrow,US,71.2906,-156.7887,28.42,93,100,5.75,mist,2022-06-13 00:50:07


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))# 2. Prompt the user to enter minimum and maximum temperature criteria 


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Kieta,PG,-6.2167,155.6333,85.26,66,18,5.68,few clouds,2022-06-13 00:50:07
3,3,Adrar,MR,20.5022,-10.0711,89.20,26,1,11.90,clear sky,2022-06-13 00:50:07
5,5,Bac Lieu,VN,9.2850,105.7244,82.67,77,100,6.22,overcast clouds,2022-06-13 00:50:08
18,18,Avarua,CK,-21.2078,-159.7750,80.65,65,100,10.36,overcast clouds,2022-06-13 00:50:12
20,20,Tamandare,BR,-8.7597,-35.1047,75.85,86,6,3.04,clear sky,2022-06-13 00:50:12
22,22,Mulege,MX,26.8833,-111.9833,89.55,46,6,4.72,clear sky,2022-06-13 00:50:13
26,26,Sorong,ID,-0.8833,131.2500,82.54,74,98,6.49,overcast clouds,2022-06-13 00:50:14
32,32,Laguna,US,38.4210,-121.4238,83.75,55,0,18.41,clear sky,2022-06-13 00:50:16
35,35,Kapaa,US,22.0752,-159.3190,85.98,77,75,14.97,broken clouds,2022-06-13 00:50:17
39,39,Atuona,PF,-9.8000,-139.0333,77.58,66,95,18.25,overcast clouds,2022-06-13 00:50:18


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))

204


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))

204


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kieta,PG,85.26,few clouds,-6.2167,155.6333,
3,Adrar,MR,89.20,clear sky,20.5022,-10.0711,
5,Bac Lieu,VN,82.67,overcast clouds,9.2850,105.7244,
18,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,
20,Tamandare,BR,75.85,clear sky,-8.7597,-35.1047,
22,Mulege,MX,89.55,clear sky,26.8833,-111.9833,
26,Sorong,ID,82.54,overcast clouds,-0.8833,131.2500,
32,Laguna,US,83.75,clear sky,38.4210,-121.4238,
35,Kapaa,US,85.98,broken clouds,22.0752,-159.3190,
39,Atuona,PF,77.58,overcast clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kieta,PG,85.26,few clouds,-6.2167,155.6333,Uruna Bay Retreat
3,Adrar,MR,89.20,clear sky,20.5022,-10.0711,
5,Bac Lieu,VN,82.67,overcast clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
18,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,Paradise Inn
20,Tamandare,BR,75.85,clear sky,-8.7597,-35.1047,Pousada 100 Passos
22,Mulege,MX,89.55,clear sky,26.8833,-111.9833,Casa Granada
26,Sorong,ID,82.54,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
32,Laguna,US,83.75,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
35,Kapaa,US,85.98,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
39,Atuona,PF,77.58,overcast clouds,-9.8000,-139.0333,Villa Enata


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:

# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
display(fig)

Figure(layout=FigureLayout(height='420px'))